In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# Enter the ID of the Google Drive folder containing train.csv and test.csv files.
# The ID of the folder is the long string of numbers and letters in the URL of the folder in Google Drive.

file_list = drive.ListFile({'q': "'1V76PUZB9NA-kWRJmJdDGbtnCidNWpT9C' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: HousingPrices.ipynb, id: 1Bi7AmvHdffrqgVLjBNCdHtSbPYzT44Wp
title: train (1), id: 1_2k3JMa33JeuO29WjNq8Tvz15ySqG1sVUUQmEQCM6yg
title: test (1).csv, id: 12g-Ro232gNbmTHKW9z3_fFDKrLpEJRTQ
title: train (1).csv, id: 1ygU2lQVVRTE1x_kMXD7MCNb1q7irc2ts


In [0]:
# Enter the ID of train.csv and test.csv. The IDs are printed in the output of the cell above.
train_downloaded = drive.CreateFile({'id': '1ygU2lQVVRTE1x_kMXD7MCNb1q7irc2ts'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '12g-Ro232gNbmTHKW9z3_fFDKrLpEJRTQ'})
test_downloaded.GetContentFile('test.csv')

In [0]:
# See the top rows of the train.csv file
!ls
!head train.csv
!head test.csv
#!wc -l train.csv


adc.json		 training_examples11.tfr  training_examples5.tfr
model_dir		 training_examples12.tfr  training_examples6.tfr
sample_data		 training_examples13.tfr  training_examples7.tfr
test.csv		 training_examples1.tfr   training_examples8.tfr
train.csv		 training_examples2.tfr   training_examples9.tfr
training_examples0.tfr	 training_examples3.tfr
training_examples10.tfr  training_examples4.tfr


In [0]:
# Import pandas, which is the library for the data structures being used
import tensorflow as tf
import pandas as pd
from dateutil import parser
import datetime
from datetime import date
import math

# Load train.csv into pandas dataframe and print the summary
df = pd.read_csv('train.csv')
df1 = pd.read_csv('test.csv')


In [0]:
# Show shapes of the data
print ("Train data shape:", df.shape)
print ("Test data shape:", df1.shape)
print list(df)
print list(df1)

import tensorflow as tf
import pandas as pd
from dateutil import parser
import datetime
from datetime import date
import math



def get_features(row):
  complete = True
  SalePrice = row['SalePrice']
  OverallQual = row['OverallQual']
  OverallCond = row['OverallCond']
  YearBuilt = row['YearBuilt']
  FullBath = row['FullBath']
  BedroomAbvGr = row['BedroomAbvGr']
  HalfBath = row['HalfBath']
  LotArea = row['LotArea']
  GrLivArea = row['GrLivArea']
  Neighborhood = row['Neighborhood']
  if math.isnan(SalePrice) \
    or math.isnan(OverallQual) \
    or math.isnan(OverallCond) \
    or math.isnan(YearBuilt) \
    or math.isnan(FullBath) \
    or math.isnan(BedroomAbvGr):
    complete = False
  features = {
      "SalePrice": tf.train.Feature(float_list = tf.train.FloatList(value=[math.log(SalePrice)])),
      "OverallQual": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallQual])),
      "OverallCond": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallCond])),
      "YearBuilt": tf.train.Feature(float_list = tf.train.FloatList(value=[(YearBuilt - 1990.) / 10.])),
      "FullBath": tf.train.Feature(float_list = tf.train.FloatList(value=[FullBath])),
      "BedroomAbvGr": tf.train.Feature(float_list = tf.train.FloatList(value=[BedroomAbvGr])),
      "HalfBath": tf.train.Feature(float_list = tf.train.FloatList(value=[HalfBath])),
      "LotArea": tf.train.Feature(float_list = tf.train.FloatList(value=[LotArea / 1000.0])),
      "GrLivArea": tf.train.Feature(float_list = tf.train.FloatList(value=[GrLivArea / 1000.0])),
      "Neighborhood": tf.train.Feature(bytes_list = tf.train.BytesList(value=[Neighborhood.encode("utf8")])),
  }
  return (features, complete)

reader = pd.read_csv('train.csv', iterator=True)
counter = 0
for j in range(14):
  tfr_file = ("training_examples%d.tfr" % j)
  print ("generating TFRecord file " + tfr_file)
  writer = tf.python_io.TFRecordWriter(tfr_file)

  # For int features:
  # 'name': tf.train.Feature(int64_list = tf.train.Int64List(value = ...))
  # For float features:
  # 'name': tf.train.Feature(float_list = tf.train.FloatList(value = ...))

  for i in range(10):
    print ("processing chunk #" + str(i))
    df = reader.get_chunk(10)

    for index, row in df.iterrows():
      feature = get_features(row)
      if feature[1]:
        counter=counter+1
        example = tf.train.Example(
            features=tf.train.Features(feature=feature[0]))
        writer.write(example.SerializeToString())
        #print example

  writer.close()
print(counter)


In [0]:
!ls -l

In [0]:
feature_columns = [
    tf.feature_column.numeric_column('OverallQual', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('OverallCond', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('YearBuilt', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('FullBath', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('BedroomAbvGr', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('HalfBath', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('LotArea', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('GrLivArea', shape=(1,), dtype=tf.float32),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_hash_bucket(
        'Neighborhood', hash_bucket_size=1000),
        dimension=1)
]
print(feature_columns)
label_column = tf.feature_column.numeric_column('SalePrice', shape=(1,), dtype=tf.float32)

features_spec = tf.feature_column.make_parse_example_spec(
    feature_columns + [label_column]
)

In [0]:
def input_fn(file_list):
  dataset = tf.contrib.data.make_batched_features_dataset(
      file_pattern=file_list,
      batch_size=100,
      features=features_spec,
      num_epochs=100,
      shuffle=True,
      shuffle_buffer_size=10
  )
  it = dataset.make_one_shot_iterator()
  features = it.get_next()
  labels = features.pop('SalePrice')
  return features, labels


In [0]:
!rm -r model_dir

In [0]:
model_dir="model_dir"

regressor = tf.estimator.DNNRegressor(
    hidden_units=[3],
    feature_columns=feature_columns,
    model_dir=model_dir
)

def get_eval_metrics(labels, predictions):
    print("hi")
    print(tf.metrics.root_mean_squared_error(
        labels=labels,
        predictions=predictions['predictions']
    ))
    return {
      'rmse': tf.metrics.root_mean_squared_error(
          labels=labels,
          predictions=predictions['predictions']
      )
  }

regressor = tf.contrib.estimator.add_metrics(regressor, get_eval_metrics)

In [0]:
train_spec = tf.estimator.TrainSpec(
    input_fn=lambda: input_fn([
        'training_examples1.tfr',
        'training_examples2.tfr',
        'training_examples3.tfr',
        'training_examples4.tfr',
        'training_examples5.tfr',
        'training_examples16tfr',
        'training_examples7.tfr',
        'training_examples8.tfr',
        'training_examples9.tfr',
        'training_examples10.tfr']), #,'training_examples2.tfr','training_examples3.tfr','training_examples4.tfr','training_examples5.tfr','training_examples6.tfr','training_examples7.tfr','training_examples8.tfr','training_examples9.tfr']),
    max_steps=1000000
)

# For evaluation, don't shuffle, and read evaluation data just once.
def eval_input_fn(file_list):
  dataset = tf.contrib.data.make_batched_features_dataset(
      file_pattern=file_list,
      batch_size=100,
      features=features_spec,
      num_epochs=1,
      shuffle=False
      #shuffle_buffer_size=10
  )
  it = dataset.make_one_shot_iterator()
  features = it.get_next()
  labels = features.pop('SalePrice')
  return features, labels

eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda: eval_input_fn([
        'training_examples0.tfr', 
        'training_examples11.tfr',
        'training_examples12.tfr',
        'training_examples13.tfr'])
)


In [0]:
counter = 0

while counter < 50:
  tf.estimator.train_and_evaluate(regressor, train_spec, eval_spec)
  counter = counter + 1
  print(counter)

In [0]:
variable_names = regressor.get_variable_names()
print ("variable names: ")
for var_name in variable_names:
  values = regressor.get_variable_value(var_name)
  print (var_name, values)


def get_test_features(row):
  
  OverallQual = row['OverallQual']
  OverallCond = row['OverallCond']
  YearBuilt = row['YearBuilt']
  FullBath = row['FullBath']
  BedroomAbvGr = row['BedroomAbvGr']
  HalfBath = row['HalfBath']
  LotArea = row['LotArea']
  GrLivArea = row['GrLivArea']
  Neighborhood = row['Neighborhood']

  features = {
      "OverallQual": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallQual])),
      "OverallCond": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallCond])),
      "YearBuilt": tf.train.Feature(float_list = tf.train.FloatList(value=[(YearBuilt - 1990) / 10])),
      "FullBath": tf.train.Feature(float_list = tf.train.FloatList(value=[FullBath])),
      "BedroomAbvGr": tf.train.Feature(float_list = tf.train.FloatList(value=[BedroomAbvGr])),
      "HalfBath": tf.train.Feature(float_list = tf.train.FloatList(value=[HalfBath])),
      "LotArea": tf.train.Feature(float_list = tf.train.FloatList(value=[LotArea / 1000.0])),
      "GrLivArea": tf.train.Feature(float_list = tf.train.FloatList(value=[GrLivArea / 1000.0])),
      "Neighborhood": tf.train.Feature(bytes_list = tf.train.BytesList(value=[Neighborhood.encode("utf8")])),
  }
  return features

In [0]:
import itertools
import math

reader = pd.read_csv('test.csv', iterator = True)
testdf = reader.get_chunk(1459)


testdf['YearBuilt'] = (testdf['YearBuilt'] - 1990) / 10.0
testdf['LotArea'] = testdf['LotArea'] / 1000.
testdf['GrLivArea'] = testdf['GrLivArea'] / 1000.

for index, row in testdf.iterrows():
  OverallQual = row['OverallQual']
  OverallCond = row['OverallCond']
  YearBuilt = row['YearBuilt']
  FullBath = row['FullBath']
  BedroomAbvGr = row['BedroomAbvGr']
  HalfBath = row['HalfBath']
  LotArea = row['LotArea']
  GrLivArea = row['GrLivArea']
  Neighborhood = row['Neighborhood']
  if math.isnan(OverallQual):
    row['OverallQual'] = 5
  if math.isnan(OverallCond):
    row['OverallCond'] = 5
  if math.isnan(YearBuilt):
    row['YearBuilt'] = 0
  if math.isnan(FullBath):
    row['FullBath'] = 2
  if math.isnan(BedroomAbvGr):
    row['BedroomAbvGr'] = 2
    
def eval_input_fn1():
  eval_dataset1 = tf.data.Dataset.from_tensor_slices((dict(reader)))
  #eval_dataset1 = eval_dataset1.shuffle(batch_size).repeat().batch(batch_size)
  eval_dataset1 = eval_dataset1.batch(len(reader))
  return eval_dataset1

def test_input_fn(df):
  x = {
      'OverallQual':tf.constant(
      df['OverallQual'].values,
          shape = [df['OverallQual'].size,1],
          dtype=tf.float32),
      'OverallCond':tf.constant(
      df['OverallCond'].values,
          shape = [df['OverallCond'].size,1],
          dtype=tf.float32),
      'YearBuilt':tf.constant(
      df['YearBuilt'].values,
          shape = [df['YearBuilt'].size,1],
          dtype=tf.float32),
      'FullBath':tf.constant(
      df['FullBath'].values,
          shape = [df['FullBath'].size,1],
          dtype=tf.float32),
      'BedroomAbvGr':tf.constant(
      df['BedroomAbvGr'].values,
          shape = [df['BedroomAbvGr'].size,1],
          dtype=tf.float32),
      'HalfBath':tf.constant(
      df['HalfBath'].values,
          shape = [df['HalfBath'].size,1],
          dtype=tf.float32),
      'LotArea':tf.constant(
      df['LotArea'].values,
          shape = [df['LotArea'].size,1],
          dtype=tf.float32),
      'GrLivArea':tf.constant(
      df['GrLivArea'].values,
          shape = [df['GrLivArea'].size,1],
          dtype=tf.float32),
      'Neighborhood':tf.constant(
      df['Neighborhood'].values,
          shape = [df['Neighborhood'].size,1],
          dtype=tf.string)
  }
  return x

def test_fn_predictions():
  return test_input_fn(testdf)

# For int features:
# 'name': tf.train.Feature(int64_list = tf.train.Int64List(value = ...))
# For float features:
# 'name': tf.train.Feature(float_list = tf.train.FloatList(value = ...))

predictions_generator = regressor.predict(
    input_fn=test_fn_predictions,
    yield_single_examples=True)

iter = itertools.islice(predictions_generator, 1459)

predictions = [y['predictions'][0] for y in iter]

print len(predictions), "predictions:"
print predictions

predictions = [math.exp(i) for i in predictions]
submission = [["Id", "SalePrice"]]
for i in range(1459):
  submission.append([testdf['Id'][i], predictions[i]])
print submission

#   # Print the prediction results.
# print("\nPrediction results:")
# print predict_results
# print next(predict_results)
# for i, prediction in enumerate(predict_results):
#   print prediction